In [ ]:
%pip install tensorflow-gpu
%pip install tensorflow_hub #TensorFlow Hub is a repository of trained machine learning models.

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import matplotlib.pylab as plt
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np

In [ ]:
print(hub.__version__)

0.15.0


In [ ]:
import pandas as pd
pd.set_option("display.precision", 8)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Set your training folder which contains images

[Drive Link](https://drive.google.com/drive/folders/1kdwUgvyqoN5JZD6q9T1YH0OKChH64Vtd?usp=share_link)

P.S: You can use shoe folder from this [Git repo](https://github.com/rfatcakr/Tensorflow_object_training/tree/master/sample_inputs).

In [ ]:
data_root = '/content/drive/MyDrive/10%'

In [ ]:
IMAGE_SHAPE = (224, 224)
TRAINING_DATA_DIR = str(data_root)
print(TRAINING_DATA_DIR);
datagen_kwargs = dict(rescale=1./255, validation_split=.20)
valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**datagen_kwargs)
valid_generator = valid_datagen.flow_from_directory(
    TRAINING_DATA_DIR,
    subset="validation",
    shuffle=True,
    target_size=IMAGE_SHAPE,
    color_mode="rgb"
)

train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**datagen_kwargs)
train_generator = train_datagen.flow_from_directory(
    TRAINING_DATA_DIR,
    subset="training",
    shuffle=True,
    target_size=IMAGE_SHAPE,
    color_mode="rgb"
    )


/content/drive/MyDrive/10%
Found 474 images belonging to 2 classes.
Found 1898 images belonging to 2 classes.


In [ ]:
image_batch_train, label_batch_train = next(iter(train_generator))
print("Image batch shape: ", image_batch_train.shape)
print("Label batch shape: ", label_batch_train.shape)
dataset_labels = sorted(train_generator.class_indices.items(), key=lambda pair:pair[1])
dataset_labels = np.array([key.title() for key, value in dataset_labels])
print(dataset_labels)

Image batch shape:  (32, 224, 224, 3)
Label batch shape:  (32, 2)
['Not_Stress' 'Stress']


In [ ]:
import tensorflow as tf

In [ ]:
#regnety
model_regnety = tf.keras.Sequential([
    hub.KerasLayer("https://tfhub.dev/google/imagenet/inception_v1/classification/5")
])
model_regnety.build([None, 224, 224, 3])  # Batch input shape.
model_regnety.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1001)              6633209   
                                                                 
Total params: 6633209 (25.30 MB)
Trainable params: 0 (0.00 Byte)
Non-trainable params: 6633209 (25.30 MB)
_________________________________________________________________


In [ ]:
model_regnety.add(tf.keras.layers.Dense(2, activation = 'softmax'))

In [ ]:
from tensorflow import keras

In [ ]:
METRICS = [
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'),
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
      keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve

]

model_regnety.compile(
  optimizer=tf.keras.optimizers.Adam(),
  loss='categorical_crossentropy',
  metrics=METRICS)

In [ ]:

steps_per_epoch = np.ceil(train_generator.samples/train_generator.batch_size)
val_steps_per_epoch = np.ceil(valid_generator.samples/valid_generator.batch_size)

hist = model_regnety.fit(
    train_generator,
    epochs=1,
    verbose=1,
    steps_per_epoch=steps_per_epoch,
    validation_data=valid_generator,
    validation_steps=val_steps_per_epoch).history

# steps_per_epoch = np.ceil(train_generator.samples/train_generator.batch_size)
# val_steps_per_epoch = np.ceil(valid_generator.samples/valid_generator.batch_size)

# hist = model_regnety.fit(
#     train_generator,
#     epochs=10,
#     verbose=1,
#     steps_per_epoch=steps_per_epoch,
#     validation_data=valid_generator,
#     validation_steps=val_steps_per_epoch).history

60/60 [==============================] - 552s 9s/step - loss: 0.5714 - tp: 1370.0000 - fp: 528.0000 - tn: 1370.0000 - fn: 528.0000 - accuracy: 0.7218 - precision: 0.7218 - recall: 0.7218 - auc: 0.7941 - prc: 0.7877 - val_loss: 0.5150 - val_tp: 367.0000 - val_fp: 107.0000 - val_tn: 367.0000 - val_fn: 107.0000 - val_accuracy: 0.7743 - val_precision: 0.7743 - val_recall: 0.7743 - val_auc: 0.8481 - val_prc: 0.8424


In [ ]:
# saved_model_path = 'my_model_3nov.h5' # or you can simply use 'my_mode.h5'
from tensorflow.keras.models import load_model, save_model

model_regnety.save('my_model_4nov.keras') #save your model

In [ ]:
from google.colab import files
files.download('my_model_4nov.h5')


In [ ]:
# model_regnety.predict('/content/drive/MyDrive/10%/stress/33422.jpg')
import cv2
import numpy as np

# Load the image
image = cv2.imread('/content/drive/MyDrive/10%/stress/33422.jpg')

# Preprocess the image (resize, normalize, etc.)
# Ensure that it matches the expected input shape of your model
image = cv2.resize(image, (224, 224))  # Replace 'height' and 'width' with the appropriate values
image = image / 255.0  # Normalize pixel values to [0, 1]

# Reshape the image to match the expected input shape (add batch dimension)
input_data = np.expand_dims(image, axis=0)

# Make predictions
predictions = model_regnety.predict(input_data)
print(predictions)


In [ ]:
import matplotlib.pyplot as plt

# Assuming you have a list of class labels
class_labels = ["Class 0", "Class 1"]

# Assuming 'predictions' is a numpy array with class probabilities
predicted_classes = np.argmax(predictions, axis=1)

for i in range(len(predictions)):
    plt.imshow(input_data[i])  # Display the input image
    plt.title(f'Predicted Class: {class_labels[predicted_classes[i]]}')
    plt.show()


In [ ]:
import numpy as np

# Assuming 'predictions' is a numpy array with class probabilities
# and 'class_labels' is a list of class labels
predicted_classes = np.argmax(predictions, axis=1)

# Get the corresponding class labels
predicted_labels = [class_labels[i] for i in predicted_classes]

# Now, 'predicted_labels' contains the class labels for each prediction
print(predicted_labels)

In [ ]:
from tensorflow import keras

In [ ]:
# to reload your model
# shoe_model = keras.models.load_model(saved_model_path)
shoe_model = tf.keras.models.load_model(
       ("/content/MY_MODE"),
       custom_objects={'KerasLayer':hub.KerasLayer}
)

In [ ]:
# Get images and labels batch from validation dataset generator

val_image_batch, val_label_batch = next(iter(valid_generator))
true_label_ids = np.argmax(val_label_batch, axis=-1)

print("Validation batch shape:", val_image_batch.shape)

In [ ]:
tf_model_predictions = shoe_model.predict(val_image_batch)
print("Prediction results shape:", tf_model_predictions.shape)

In [ ]:
tf_pred_dataframe = pd.DataFrame(tf_model_predictions)
tf_pred_dataframe.columns = dataset_labels

print("Prediction results for the first elements")
tf_pred_dataframe.head()

In [ ]:
predicted_ids = np.argmax(tf_model_predictions, axis=-1)
predicted_labels = dataset_labels[predicted_ids]

In [ ]:
plt.figure(figsize=(10,9))
plt.subplots_adjust(hspace=0.5)
for n in range(30):
  plt.subplot(6,5,n+1)
  plt.imshow(val_image_batch[n])
  color = "green" if predicted_ids[n] == true_label_ids[n] else "red"
  plt.title(predicted_labels[n].title(), color=color)
  plt.axis('off')
_ = plt.suptitle("Model predictions (green: correct, red: incorrect)")

<img src="./results/only-stress-results.jpg" style="width:500px;height:500px">

In [ ]:
import shutil
shutil.make_archive('MY_MODE.zip', 'zip', 'MY_MODE')